<a href="https://colab.research.google.com/github/sergiolopezzl/prueba-llm-rag/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [61]:
!pip install -U \
  langchain \
  langchain-core \
  langchain-community \
  langchain-google-vertexai \
  google-cloud-aiplatform \
  google-cloud-storage \
  chromadb


In [62]:
import os
import vertexai

os.environ["GCP_PROJECT_ID"] = "sandbox-cigo"
os.environ["REGION"] = "us-central1"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "./cred.json"

vertexai.init(
    project=os.getenv("GCP_PROJECT_ID"),
    location=os.getenv("REGION")
)

print("✅ Vertex AI inicializado correctamente")


✅ Vertex AI inicializado correctamente


In [63]:
from langchain_google_vertexai import ChatVertexAI, VertexAIEmbeddings
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.vectorstores import Chroma


In [64]:
docs = [
    Document(
        page_content="""
        Producto: Laptop Gaming ASUS ROG
        Categoria: Laptop
        Uso: Gaming
        Precio: 899 USD
        RAM: 16GB
        GPU: RTX 3060
        Rating: 4.6
        """
    ),
    Document(
        page_content="""
        Producto: Laptop Dell Inspiron
        Categoria: Laptop
        Uso: Oficina
        Precio: 650 USD
        RAM: 8GB
        GPU: Intel UHD
        Rating: 4.2
        """
    ),
    Document(
        page_content="""
        Producto: Mochila Targus 15 pulgadas
        Categoria: Accesorios
        Uso: Portátil
        Precio: 45 USD
        Rating: 4.5
        """
    ),
    Document(
        page_content="""
        Pedido: PED001
        Usuario: sergio
        Estado: En tránsito
        Fecha: 2024-01-15
        Productos: Laptop Gaming ASUS ROG, Mochila Targus
        """
    )
]


In [65]:
embeddings = VertexAIEmbeddings(
    model_name="text-embedding-004"
)

vectorstore = Chroma.from_documents(
    documents=docs,
    embedding=embeddings,
    persist_directory="./chroma_db"
)

retriever = vectorstore.as_retriever(
    search_kwargs={"k": 4}
)

print("✅ Vector Store RAG creado")


/tmp/ipython-input-1925593401.py:1: DeprecationWarning: Use [`GoogleGenerativeAIEmbeddings`][langchain_google_genai.GoogleGenerativeAIEmbeddings] instead.
  embeddings = VertexAIEmbeddings(


✅ Vector Store RAG creado


In [66]:
llm = ChatVertexAI(
    model="gemini-2.5-pro",
    temperature=0.2,
    max_output_tokens=2024
)


/tmp/ipython-input-2295163715.py:1: DeprecationWarning: Use [`ChatGoogleGenerativeAI`][langchain_google_genai.ChatGoogleGenerativeAI] instead.
  llm = ChatVertexAI(


In [67]:
intent_prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        """
        Clasifica la intención del usuario en UNA sola categoría.

        Categorías posibles:
        - busqueda_producto
        - recomendacion
        - comparacion
        - consulta_pedido
        - soporte
        - otro

        Responde SOLO con el nombre de la categoría.
        """
    ),
    ("human", "{question}")
])


In [68]:
prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        """
        Eres DaviBot, el asistente virtual de ecommerce del Banco Davivienda.

        Lineamientos de comunicación:
        - Tono cercano, confiable y profesional
        - Respuestas completas y bien estructuradas
        - No dejes frases incompletas
        - Ofrece ayuda adicional cuando sea útil

        Reglas estrictas:
        - Usa SOLO la información del contexto
        - Convierte precios a COP (USD x 3700)
        - Si no hay información suficiente, dilo con claridad
        """
    ),
    ("system", "Historial de conversación:\n{chat_history}"),
    ("system", "Contexto disponible:\n{context}"),
    ("human", "{question}")
])


In [69]:
chat_history = []


In [70]:
def classify_intent(question: str) -> str:
    messages = intent_prompt.format_messages(question=question)
    response = llm.invoke(messages).content.lower().strip()
    return response


In [71]:
def rag_agent(question: str) -> str:
    # 1️⃣ Clasificar intención
    intent = classify_intent(question)

    # 2️⃣ Recuperar contexto según intención
    retrieved_docs = retriever.invoke(question)
    context = "\n\n".join(d.page_content for d in retrieved_docs)

    # 3️⃣ Historial
    history_text = "\n".join(
        f"Usuario: {h['user']}\nDaviBot: {h['bot']}"
        for h in chat_history
    ) or "Sin historial previo"

    # 4️⃣ Prompt
    messages = prompt.format_messages(
        question=question,
        context=context,
        chat_history=history_text
    )

    # 5️⃣ LLM
    response = llm.invoke(messages).content

    # 6️⃣ Guardar historial
    chat_history.append({
        "user": question,
        "bot": response,
        "intent": intent
    })

    return f"📌 Intención detectada: {intent}\n\n{response}"


In [72]:
preguntas = [
    "Busco una laptop para gaming",
    "¿Cuál me recomiendas para trabajar y jugar?",
    "Compárame la laptop gaming con la Dell",
    "Cuál es el estado del pedido PED001",
    "Tienes mochilas para portátil?"
]

for p in preguntas:
    print("\n👤 Usuario:", p)
    print("🏦 DaviBot:\n", rag_agent(p))



👤 Usuario: Busco una laptop para gaming
🏦 DaviBot:
 📌 Intención detectada: busqueda_producto

¡Hola! Con gusto te ayudo a encontrar una excelente opción para tus sesiones de juego.

Basado en la información disponible, te recomiendo la **Laptop Gaming ASUS ROG**. Este equipo está diseñado específicamente para gaming y cuenta con características muy potentes.

Aquí te comparto sus detalles:

*   **Producto:** Laptop Gaming ASUS ROG
*   **Uso principal:** Gaming
*   **Memoria RAM:** 16GB
*   **Tarjeta gráfica (GPU):** RTX 3060
*   **Calificación de usuarios:** 4.6 sobre 5
*   **Precio:** Su valor es de 899 USD, que equivale aproximadamente a **$3,326,300 COP**.

Este equipo es una gran elección gracias a su tarjeta gráfica RTX 3060 y sus 16GB de RAM, que te asegurarán un rendimiento fluido y una gran experiencia visual en tus juegos favoritos.

¿Te gustaría conocer más detalles sobre este producto o necesitas que te ayude con algo más?

👤 Usuario: ¿Cuál me recomiendas para trabajar y ju